<a target="_blank" href="https://colab.research.google.com/github/gerakys/PyhtonProject_DMTA/blob/main/Neural_Net_FND.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Fake News Detection using Neural Networks

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [12]:
# Load the dataset
data = pd.read_csv("data/fake_or_real_news.csv")
data.head(15)

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL
5,6903,"Tehran, USA","\nI’m not an immigrant, but my grandparents ...",FAKE
6,7341,Girl Horrified At What She Watches Boyfriend D...,"Share This Baylee Luciani (left), Screenshot o...",FAKE
7,95,‘Britain’s Schindler’ Dies at 106,A Czech stockbroker who saved more than 650 Je...,REAL
8,4869,Fact check: Trump and Clinton at the 'commande...,Hillary Clinton and Donald Trump made some ina...,REAL
9,2909,Iran reportedly makes new push for uranium con...,Iranian negotiators reportedly have made a las...,REAL


## Data Preprocessing

#### ...

In [22]:
# Check for missing values
data.isnull().sum()
data.shape

(6335, 4)

In [15]:
# Extract features and labels
x = np.array(data["title"])
y = np.array(data["label"])

In [16]:
# Convert labels to numerical format
le = LabelEncoder()
y = le.fit_transform(y)

In [17]:
# Tokenize the text data
max_words = 5000
tokenizer = Tokenizer(num_words=max_words, split=' ') # definiamo il nostro tokenizer
tokenizer.fit_on_texts(x) # fittiamo sui nostri dati il tokenizer
x = tokenizer.texts_to_sequences(x) #trasformazione del testo in sequenze di interi
x = pad_sequences(x) # padding per uniformare le sequenze

In [7]:
# we take a look at our vocabulary
indice_parole = tokenizer.word_index
indice_parole

{'the': 1,
 'to': 2,
 'in': 3,
 'of': 4,
 'trump': 5,
 'for': 6,
 'on': 7,
 'a': 8,
 'and': 9,
 'is': 10,
 'clinton': 11,
 'hillary': 12,
 'with': 13,
 'obama': 14,
 'new': 15,
 'by': 16,
 'as': 17,
 'donald': 18,
 'from': 19,
 'at': 20,
 'why': 21,
 'gop': 22,
 'election': 23,
 'us': 24,
 'how': 25,
 'are': 26,
 'will': 27,
 '2016': 28,
 'be': 29,
 'about': 30,
 'what': 31,
 'video': 32,
 'after': 33,
 'over': 34,
 '–': 35,
 'it': 36,
 'not': 37,
 'this': 38,
 'that': 39,
 'debate': 40,
 'campaign': 41,
 'says': 42,
 'has': 43,
 'up': 44,
 'war': 45,
 'his': 46,
 'house': 47,
 'fbi': 48,
 'state': 49,
 'news': 50,
 'russia': 51,
 'out': 52,
 'you': 53,
 'sanders': 54,
 's': 55,
 'u': 56,
 'white': 57,
 'more': 58,
 'against': 59,
 'no': 60,
 'just': 61,
 'have': 62,
 'isis': 63,
 'could': 64,
 'world': 65,
 'an': 66,
 'comment': 67,
 'president': 68,
 'iran': 69,
 'america': 70,
 'cruz': 71,
 'republicans': 72,
 'can': 73,
 'police': 74,
 'all': 75,
 'was': 76,
 'deal': 77,
 'but': 78

## Let's Build our Model
#### ...

In [18]:
# Split the data into training and testing sets
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.2, random_state=42)

In [26]:
data['label'].value_counts()

REAL    3171
FAKE    3164
Name: label, dtype: int64

In [19]:
# Build the neural network model
model = Sequential() # creiamo un modello sequenziale, strato per strati
model.add(Embedding(max_words, 128, input_length=x.shape[1])) # layer di embedding
model.add(SpatialDropout1D(0.2)) # per prevenire l'overfitting si disattivano il 20% dei precedenti embeddings
model.add(LSTM(100)) # Tipo di RNN. il valore tra parentesi indica il numero di nodi
model.add(Dense(1, activation='sigmoid')) # layer "fully connected" denso che termina il modello con un singolo neurone 
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) # funzione di loss, di ottimizzazione e metrica di valutazione

In [23]:
# Train the model
batch_size = 32 # provare a cambiare 
epochs = 60 # prova 60
model.fit(xtrain, ytrain, epochs=epochs, batch_size=batch_size, validation_split=0.2)

Epoch 1/60
127/127 [==============================] - 7s 58ms/step - loss: 0.1022 - accuracy: 0.9640 - val_loss: 0.6158 - val_accuracy: 0.7978
Epoch 2/60
127/127 [==============================] - 7s 57ms/step - loss: 0.0569 - accuracy: 0.9808 - val_loss: 0.7298 - val_accuracy: 0.7870
Epoch 3/60
127/127 [==============================] - 7s 57ms/step - loss: 0.0684 - accuracy: 0.9761 - val_loss: 0.7146 - val_accuracy: 0.8047
Epoch 4/60
127/127 [==============================] - 7s 55ms/step - loss: 0.0270 - accuracy: 0.9921 - val_loss: 0.8344 - val_accuracy: 0.8047
Epoch 5/60
127/127 [==============================] - 7s 55ms/step - loss: 0.0148 - accuracy: 0.9963 - val_loss: 0.8967 - val_accuracy: 0.7949
Epoch 6/60
127/127 [==============================] - 7s 56ms/step - loss: 0.0074 - accuracy: 0.9983 - val_loss: 0.9992 - val_accuracy: 0.7890
Epoch 7/60
127/127 [==============================] - 7s 55ms/step - loss: 0.0056 - accuracy: 0.9990 - val_loss: 1.0876 - val_accuracy: 0.7939

Epoch 57/60
127/127 [==============================] - 8s 61ms/step - loss: 6.6325e-05 - accuracy: 1.0000 - val_loss: 1.9113 - val_accuracy: 0.7909
Epoch 58/60
127/127 [==============================] - 8s 63ms/step - loss: 0.0173 - accuracy: 0.9948 - val_loss: 1.1569 - val_accuracy: 0.7821
Epoch 59/60
127/127 [==============================] - 8s 61ms/step - loss: 0.0044 - accuracy: 0.9988 - val_loss: 1.3160 - val_accuracy: 0.7840
Epoch 60/60
127/127 [==============================] - 7s 56ms/step - loss: 7.2433e-04 - accuracy: 1.0000 - val_loss: 1.4650 - val_accuracy: 0.7959


In [29]:
# Evaluate the model on the test set

loss, accuracy = model.evaluate(xtest, ytest)
print(f"Model Accuracy: {accuracy:.2f}")
print(f"Model Loss: {loss}")

40/40 [==============================] - 1s 18ms/step - loss: 1.5687 - accuracy: 0.7790
Model Accuracy: 0.78
Model Loss: 1.5687307119369507


In [28]:
# Make predictions on a sample news headline
news_headline = input("Type News title here: ")
headline_seq = tokenizer.texts_to_sequences(news_headline)
headline_padded = pad_sequences(headline_seq, maxlen=x.shape[1])
result = model.predict(headline_padded)[0][0]
predicted_label = "Real" if result < 0.5 else "Fake"
print(f"Predicted Label: {predicted_label} (Probability: {result:.2f})")

Type News title here: Will Biden ever stand up to Benjamin Netanyahu? Don’t bet on it
2/2 [==============================] - 1s 17ms/step
Predicted Label: Real (Probability: 0.00)
